In [1]:
import numpy as np
import pandas as pd
import cv2
import math
import shapely
import shapely.geometry
import shapely.geometry.polygon

INPUT_POINT_CLOUDS = [
    '/home/mitchell/sample_point_clouds/path1.pcd',
    #'/home/mitchell/sample_point_clouds/rolling.pcd',
    '/home/mitchell/sample_point_clouds/rolling2.pcd',
    '/home/mitchell/sample_point_clouds/shade1.pcd',
    '/home/mitchell/sample_point_clouds/shade2.pcd',
    #'/home/mitchell/sample_point_clouds/sun.pcd',
    #'/home/mitchell/sample_point_clouds/tree.pcd',
    '/home/mitchell/sample_point_clouds/indoors.pcd',
]

INPUT_POINT_CLOUD_IDENTIFIERS = [
    'path1',
    #'rolling',
    'rolling2',
    'shade1',
    'shade2',
    #'sun',
    #'tree',
    'indoors',
]

INPUT_POINT_CLOUD_CLICKED_FILES = [
    '/home/mitchell/sample_point_clouds/path1_clicked.txt',
    #'/home/mitchell/sample_point_clouds/rolling_clicked.txt',
    '/home/mitchell/sample_point_clouds/rolling2_clicked.txt',
    '/home/mitchell/sample_point_clouds/shade1_clicked.txt',
    '/home/mitchell/sample_point_clouds/shade2_clicked.txt',
    #'/home/mitchell/sample_point_clouds/sun_clicked.txt',
    #'/home/mitchell/sample_point_clouds/tree_clicked.txt',
    '/home/mitchell/sample_point_clouds/indoors_clicked.txt',
]

if (len(INPUT_POINT_CLOUDS) != len(INPUT_POINT_CLOUD_IDENTIFIERS)):
    raise ValueError( \
        'Mismatch in constant lengths. INPUT_POINT_CLOUDS = {0}, INPUT_POINT_CLOUD_IDENTIFIERS = {1}' \
            .format(len(INPUT_POINT_CLOUDS), len(INPUT_POINT_CLOUD_IDENTIFIERS)))

if (len(INPUT_POINT_CLOUDS) != len(INPUT_POINT_CLOUD_CLICKED_FILES)):
    raise ValueError( \
        'Mismatch in constant lengths. INPUT_POINT_CLOUDS = {0}, INPUT_POINT_CLOUD_CLICKED_FILES = {1}' \
            .format(len(INPUT_POINT_CLOUDS), len(INPUT_POINT_CLOUD_CLICKED_FILES)))

In [2]:
def parse_point(rgb, x, y, z):
    if (not (math.isfinite(rgb) and math.isfinite(x) and math.isfinite(y) and math.isfinite(z))):
        return None
    
    point = {}
    
    point['r'] = ((rgb & 0x00FF0000) >> 16)
    point['g'] = ((rgb & 0x0000FF00) >> 8)
    point['b'] = ((rgb & 0x000000FF))
    point['x'] = x
    point['y'] = y
    point['z'] = z
    
    return point

def read_pcd(pcd_file):
    line_number = 0
    points = []
    with open(pcd_file, 'r') as f:
        for line in f:
            line_number += 1
            if (line_number <= 10):
                continue
            
            line_data = line.split(' ')
            point = parse_point(
                int(line_data[0]),
                float(line_data[2]),
                float(line_data[3]),
                float(line_data[4]))
            
            if (point is not None):
                points.append(point)
    
    return pd.DataFrame(points)

test_pcd = read_pcd('/home/mitchell/sample_point_clouds/path1.pcd')
test_pcd.head()

,r,g,b,x,y,z
0,164,168,147,8.375642,8.567211,0.164210
1,199,204,179,8.354388,8.532598,0.163793
2,220,224,200,8.615615,8.786121,0.168915
3,196,201,174,8.783773,8.944074,0.172211
4,155,160,131,8.687432,8.832588,0.170322


In [3]:
def read_clicked_points(clicked_point_file):
    line_number = 0
    clicked_points = []
    
    with open(clicked_point_file, 'r') as f:
        for line in f:
            line_number += 1
            if (line_number <= 4):
                continue
            
            if (not (line.startswith('Point index picked:'))):
                raise ValueError('Unexpected data at line {0}. Wrong line prefix.'.format(line_number))
            
            open_bracket_index = line.find('[')
            close_bracket_index = line.find(']')
            
            if (open_bracket_index == -1):
                raise ValueError('Missing open bracket at line {0}.'.format(line_number))
            if (close_bracket_index == -1):
                raise ValueError('Missing close bracket at line {0}.'.format(line_number))
            
            xyz_str = line[open_bracket_index+1:close_bracket_index]
            xyz = [float(v.strip()) for v in xyz_str.split(',')]
            
            if (len(xyz) != 3):
                raise ValueError('Attempted to split {0} on comma, but did not get 3 parts.'.format(xyz_str))
            
            point = {}
            
            point['x'] = xyz[0]
            point['y'] = xyz[1]
            point['z'] = xyz[2]
            
            clicked_points.append(point)
    
    return pd.DataFrame(clicked_points)

test_clicked_points = read_clicked_points('/home/mitchell/sample_point_clouds/path1_clicked.txt')
print(test_clicked_points.shape)
test_clicked_points.head()

(7, 3)


,x,y,z
0,4.818083,0.031028,-0.560628
1,4.802900,0.159380,-0.551557
2,4.746237,0.211175,-0.473561
3,4.946481,0.255113,-0.568046
4,5.065692,0.260800,-0.566368


In [4]:
def filter_points_by_shape(point_cloud, clicked_points):
    poly = shapely.geometry.polygon.Polygon(zip(clicked_points['x'], clicked_points['y']))
    
    point_cloud['in_bounds'] = point_cloud.apply( \
        lambda r: poly.contains(shapely.geometry.point.Point(r['x'], r['y'])), axis=1)
    
    return point_cloud

test_pcd_filtered = filter_points_by_shape(test_pcd, test_clicked_points)
print(test_pcd_filtered.shape)
test_pcd_filtered.head()
print(test_pcd_filtered.groupby(by=['in_bounds']).size())

(830572, 7)
in_bounds
False    829354
True       1218
dtype: int64


In [5]:
def write_pcd(point_cloud, out_file):
    with open(out_file, 'w') as f:
        f.write('VERSION .7\n')                                                                                                                                                                                           
        f.write('FIELDS rgb x y z\n')                                                                                                                                                                    
        f.write('SIZE 4 4 4 4\n')                                                                                                                                                                    
        f.write('TYPE U F F F\n')                                                                                                                                                                                        
        f.write('COUNT 1 1 1 1\n')                                                                                                                                                                                       
        f.write('WIDTH {0}\n'.format(point_cloud.shape[0]))                                                                                                                                                                                        
        f.write('HEIGHT 1\n')                                                                                                                                                                                               
        f.write('VIEWPOINT 0 0 0 1 0 0 0\n')                                                                                                                                                                                     
        f.write('POINTS {0}\n'.format(point_cloud.shape[0]))                                                                                                                                                                                               
        f.write('DATA ASCII\n')
        
        for idx, row in point_cloud.iterrows():
            rgb = 0
            rgb |= (row['r'] << 16)
            rgb |= (row['g'] << 8)
            rgb |= (row['b'])
            
            f.write('{0} {1} {2} {3}\n'.format(rgb, row['x'], row['y'], row['z']))    
            
test_filtered = test_pcd_filtered[test_pcd_filtered['in_bounds'] == True].copy()
print(test_filtered.shape)
write_pcd(test_filtered, 'out.pcd')

(1218, 7)


In [6]:
def generate_filtering_clouds(pcd_filenames, clicked_point_filenames, identifiers):
    selected_points = pd.DataFrame(columns=['r', 'g', 'b', 'x', 'y', 'z', 'identifier'])
    unselected_points = pd.DataFrame(columns=['r', 'g', 'b', 'x', 'y', 'z', 'identifier'])
    
    for i in range(0, len(pcd_filenames), 1):
        pcd_filename = pcd_filenames[i]
        cp_filename = clicked_point_filenames[i]
        identifier = identifiers[i]
        
        print('Reading point cloud {0}...'.format(pcd_filename))
        pcd = read_pcd(pcd_filename)
        
        print('Reading clicked points {0}...'.format(cp_filename))
        cp = read_clicked_points(cp_filename)
        
        print('Generating point mappings...')
        pcd_tagged = filter_points_by_shape(pcd, cp)
        pcd_tagged['identifier'] = identifier
        
        print('Filtering and adding to list...')
        cone_points = pcd_tagged[pcd_tagged['in_bounds'] == True]
        non_cone_points = pcd_tagged[pcd_tagged['in_bounds'] == False]
        
        selected_points = selected_points.append(cone_points, ignore_index=True)
        unselected_points = unselected_points.append(non_cone_points, ignore_index=True)
        
        print('For file {0}, found {1} points in the cone (out of {2})' \
             .format(pcd_filename, cone_points.shape[0], pcd.shape[0]))
        
        cone_point_filename = '{0}_cone.pcd'.format(identifier)
        non_cone_filename = '{0}_nocone.pcd'.format(identifier)
        
        print('Writing cone points to {0}...'.format(cone_point_filename))
        write_pcd(cone_points, cone_point_filename)
        write_pcd(non_cone_points, non_cone_filename)
        
    print('In total, found {0} selected points.'.format(selected_points.shape[0]))
    print('Writing to cone_points.csv...')
    selected_points = selected_points.reset_index(drop=True)
    selected_points.to_csv('cone_points.csv', index=False, header=True)
    
    print('In total, found {0} non-selected points.'.format(unselected_points.shape[0]))
    print('Writing to non_cone_points.csv...')
    unselected_points = unselected_points.reset_index(drop=True)
    unselected_points.to_csv('non_cone_points.csv', index=False, header=True)
    
print('Generating dataset...')
generate_filtering_clouds(INPUT_POINT_CLOUDS, INPUT_POINT_CLOUD_CLICKED_FILES, INPUT_POINT_CLOUD_IDENTIFIERS)
print('Done!')

Generating dataset...
Reading point cloud /home/mitchell/sample_point_clouds/path1.pcd...
Reading clicked points /home/mitchell/sample_point_clouds/path1_clicked.txt...
Generating point mappings...
Filtering and adding to list...
For file /home/mitchell/sample_point_clouds/path1.pcd, found 1218 points in the cone (out of 830572)
Writing cone points to path1_cone.pcd...
Reading point cloud /home/mitchell/sample_point_clouds/rolling2.pcd...
Reading clicked points /home/mitchell/sample_point_clouds/rolling2_clicked.txt...
Generating point mappings...
Filtering and adding to list...
For file /home/mitchell/sample_point_clouds/rolling2.pcd, found 778 points in the cone (out of 796389)
Writing cone points to rolling2_cone.pcd...
Reading point cloud /home/mitchell/sample_point_clouds/shade1.pcd...
Reading clicked points /home/mitchell/sample_point_clouds/shade1_clicked.txt...
Generating point mappings...
Filtering and adding to list...
For file /home/mitchell/sample_point_clouds/shade1.pcd, f